In [1]:
import pandas as pd
import numpy as np
import json
    
df = pd.read_csv('../../data/imp_exp_prod_flat_all.csv')
df.drop(columns='Row Number', inplace=True)
df = df[['Area', 'Item', 'Year', 'Production', 'Import Quantity', 'Export Quantity']]
df.rename(columns={'Area': 'Country', 'Item':'Produce', 'Production':'Production Quantity'}, inplace=True)
df.head()

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
0,Armenia,Potatoes,2000,290260.0,390.0,300.0
1,Armenia,Potatoes,2001,363834.0,2290.0,0.0
2,Armenia,Potatoes,2002,374263.0,1918.0,0.0
3,Armenia,Potatoes,2003,507518.0,2853.0,7.0
4,Armenia,Potatoes,2004,576427.0,1553.0,0.0


In [2]:
# create a lookup table for countries and their centroid locations
lookup = pd.read_csv('../../data/country_centroids.csv')
lookup.rename(columns={'country':'code'}, inplace=True)
lookup.index = lookup['name']
(lookup.drop(columns=['name'])).to_csv('../../data/countries_lookup.csv')


# FIXME --------------------------------------------------------------------------
# ********************************************************************************
# Assuming that China is a sum of mainland, taiwan, hong kong and macao
# 'Serbia and Montenegro' - set coordinates to average of Serbia and Montenegro
# 'South Sudan' and 'Sudan (former)'- set coordinates to Sudan's coordinates

# print regions that are not present in the countries lookup table
set(df[~df['Country'].isin(lookup['name'])]['Country'])

{'Africa',
 'Americas',
 'Asia',
 'Australia & New Zealand',
 'Caribbean',
 'Central America',
 'Central Asia',
 'China',
 'Eastern Africa',
 'Eastern Asia',
 'Eastern Europe',
 'Europe',
 'European Union',
 'Land Locked Developing Countries',
 'Least Developed Countries',
 'Low Income Food Deficit Countries',
 'Melanesia',
 'Middle Africa',
 'Net Food Importing Developing Countries',
 'Northern Africa',
 'Northern America',
 'Northern Europe',
 'Occupied Palestinian Territory',
 'Oceania',
 'Polynesia',
 'Small Island Developing States',
 'South America',
 'South-Eastern Asia',
 'Southern Africa',
 'Southern Asia',
 'Southern Europe',
 'Western Africa',
 'Western Asia',
 'Western Europe',
 'World'}

In [3]:
# remove the regions printed above from the dataframe
df = df[df['Country'].isin(lookup.index)]

# randomly print 10 entries from the dataframe for an example of what it contains
df.sample(n=10)

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
132575,Belarus,Millet,2015,9894.0,43.0,44.0
56261,Sweden,Barley,2001,1623000.0,105217.0,322425.0
15841,Kuwait,"Chillies and peppers, green",2001,5994.0,7345.0,0.0
70989,Venezuela (Bolivarian Republic of),Carrots and turnips,2010,192252.0,0.0,3.0
36016,Czechia,Barley,2012,1616467.0,28912.0,362154.0
147822,France,Grapes,2003,6307112.0,154159.0,12128.0
36385,Czechia,Buckwheat,2007,2032.0,280.0,499.0
158462,Hungary,Sorghum,2003,6819.0,0.0,1315.0
10647,Japan,Chestnut,2015,16300.0,7083.0,294.0
68219,United States of America,Linseed,2016,219870.0,88595.0,33481.0


In [4]:
# calculate top 10 producers, importers and exporters for each produce in each year
top10_statistics = {}
top10_statistics['produce'] = produceSet = sorted(set(df['Produce']))
top10_statistics['largest_quantity'] = int((df.nlargest(1, 'Production Quantity'))['Production Quantity'])

for produce in produceSet: 
    years = sorted(set(df[df['Produce'] == produce]['Year']))
    yearStatistics = {}
    yearStatistics['available_years'] = years
    individualYears = {}
    
    for year in years:
        selected = df[(df['Produce'] == produce) & (df['Year'] == year)]
        individualYears[year] = {
            'Producers': {
                'countries': list(selected.nlargest(10, 'Production Quantity')['Country']),
                'quantities': list(selected.nlargest(10, 'Production Quantity')['Production Quantity'])
            },
            'Importers': {
                'countries': list(selected.nlargest(10, 'Import Quantity')['Country']),
                'quantities': list(selected.nlargest(10, 'Import Quantity')['Import Quantity'])
            },
            'Exporters': {
                'countries': list(selected.nlargest(10, 'Export Quantity')['Country']),
                'quantities': list(selected.nlargest(10, 'Export Quantity')['Export Quantity'])
            }
        }
        
    yearStatistics['top10_per_year'] = individualYears
    top10_statistics[produce] = yearStatistics

In [5]:
# dump top 10 statistics to a json file
json.dump(top10_statistics, open('../../data/top10_imp_exp_prod.json', 'w'))